In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gdown

In [2]:
#Helper Fuction
def import_from_gdrive(id_file, file_name):
  link = 'https://drive.google.com/uc?id=' + id_file
  output_file = '/content/' + file_name
  gdown.download(link, output_file, quiet=False)
  df = pd.read_csv(file_name)
  return df

In [3]:
link_train_combined = '1RrJ73EqxTIME-tlSAU6Ob8n5_pxRbH_V'
link_test_combined = '1HkAMyEvIKKMXKuLPXMdbV_78vM00KXX0'
train = import_from_gdrive(link_train_combined,'train.csv')
test = import_from_gdrive(link_test_combined,'test.csv')

Downloading...
From: https://drive.google.com/uc?id=1RrJ73EqxTIME-tlSAU6Ob8n5_pxRbH_V
To: /content/train.csv
100%|██████████| 78.3M/78.3M [00:00<00:00, 98.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1HkAMyEvIKKMXKuLPXMdbV_78vM00KXX0
To: /content/test.csv
100%|██████████| 25.0M/25.0M [00:00<00:00, 48.8MB/s]


In [ ]:
test.head()

In [ ]:
train.head()

In [ ]:
test.shape

In [ ]:
train.shape

In [ ]:
train.info()

#Data Preparation

In [4]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()
id_jalan_kelompok_train = train[['weekend','libur','aktivitas','cycleway','highway','sidewalk','surface']]
encoded_train = encoder.fit_transform(id_jalan_kelompok_train)

id_jalan_kelompok_test =test[['weekend','libur','aktivitas','cycleway','highway','sidewalk','surface']]
encoded_test = encoder.transform(id_jalan_kelompok_test)

In [5]:
# Ubah hasil encoding menjadi DataFrame
encoded_train_df = pd.DataFrame(encoded_train, columns=['encoded_weekend','encoded_libur','encoded_aktivitas','encoded_cycleway','encoded_highway','encoded_sidewalk','encoded_surface'])
encoded_test_df = pd.DataFrame(encoded_test, columns=['encoded_weekend','encoded_libur','encoded_aktivitas','encoded_cycleway','encoded_highway','encoded_sidewalk','encoded_surface'])

# Gabungkan hasil encoding dengan data train dan test
train_clean = pd.concat([train, encoded_train_df], axis=1)
test_clean = pd.concat([test, encoded_test_df], axis=1)

In [6]:
X_train = train_clean.drop(['weekend','libur','aktivitas','cycleway','highway','sidewalk','surface'], axis= 1)
X_test = test_clean.drop(['weekend','libur','aktivitas','cycleway','highway','sidewalk','surface'], axis= 1)
y_train = train_clean['rerata_kecepatan']

In [7]:
ID = X_test['id']
X_test = X_test.drop(['id','kelompok'], axis=1)

In [8]:
X_train = X_train.drop(['rerata_kecepatan','kelompok'], axis=1)

In [9]:
X_train["maxspeed_mph"] = X_train["maxspeed_mph"] * 1.6
X_train.rename(columns={"maxspeed_mph": "maxspeed_kph"}, inplace=True)

X_test["maxspeed_mph"] = X_test["maxspeed_mph"] * 1.6
X_test.rename(columns={"maxspeed_mph": "maxspeed_kph"}, inplace=True)

In [14]:
X_train.columns

Index(['id_jalan', 'id_titik_mulai', 'id_titik_akhir', 'titik1', 'titik2',
       'maxspeed_kph', 'lat1', 'long1', 'lat2', 'long2', 'jarak_km', 'tanggal',
       'jam', 'lanes', 'encoded_weekend', 'encoded_libur', 'encoded_aktivitas',
       'encoded_cycleway', 'encoded_highway', 'encoded_sidewalk',
       'encoded_surface'],
      dtype='object')

#CatBoost

In [ ]:
pip install catboost

In [ ]:
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score


# We define the model
model = CatBoostRegressor(objective='RMSE')

# We train model
model.fit(X_train, y_train)

Learning rate set to 0.105456
0:	learn: 7.8307421	total: 418ms	remaining: 6m 57s
1:	learn: 7.5681509	total: 639ms	remaining: 5m 19s
2:	learn: 7.3405059	total: 788ms	remaining: 4m 22s
3:	learn: 7.1435655	total: 986ms	remaining: 4m 5s
4:	learn: 6.9836624	total: 1.26s	remaining: 4m 10s
5:	learn: 6.8418221	total: 1.51s	remaining: 4m 10s
6:	learn: 6.7221989	total: 1.79s	remaining: 4m 13s
7:	learn: 6.6142953	total: 1.99s	remaining: 4m 6s
8:	learn: 6.5273919	total: 2.28s	remaining: 4m 11s
9:	learn: 6.4413484	total: 2.5s	remaining: 4m 7s
10:	learn: 6.3729910	total: 2.64s	remaining: 3m 57s
11:	learn: 6.3067533	total: 2.85s	remaining: 3m 54s
12:	learn: 6.2469864	total: 3.02s	remaining: 3m 48s
13:	learn: 6.1911709	total: 3.2s	remaining: 3m 45s
14:	learn: 6.1444231	total: 3.31s	remaining: 3m 37s
15:	learn: 6.0949712	total: 3.41s	remaining: 3m 30s
16:	learn: 6.0483484	total: 3.53s	remaining: 3m 24s
17:	learn: 6.0083076	total: 3.67s	remaining: 3m 20s
18:	learn: 5.9701979	total: 3.82s	remaining: 3m 1

In [ ]:
Y_predict = model.predict(X_test)
Y_predict

array([41.68004355, 40.34660066, 36.73585678, ..., 41.33766958,
       33.61131227, 45.32975716])

In [ ]:
output = pd.DataFrame({'id': ID,
                       'rerata_kecepatan': Y_predict})

In [ ]:
output.describe()

,id,rerata_kecepatan
count,127489.000000,127489.000000
mean,63744.000000,35.440515
std,36803.048572,7.231102
min,0.000000,7.017431
25%,31872.000000,31.429858
50%,63744.000000,35.805619
75%,95616.000000,40.289289
max,127488.000000,53.921962


In [ ]:
output.shape

(127489, 2)

In [ ]:
output.to_csv('submitteros.csv', index=False)